<a href="https://colab.research.google.com/github/msamwelmollel/presentation_RL/blob/main/cap_network2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

Functions

In [8]:
# CapsuleLayer implementation
class CapsuleLayer(layers.Layer):
    def __init__(self, num_capsules, dim_capsules, routings=3, **kwargs):
        super(CapsuleLayer, self).__init__(**kwargs)
        self.num_capsules = num_capsules
        self.dim_capsules = dim_capsules
        self.routings = routings

    def build(self, input_shape):
        self.input_num_capsules = input_shape[1]
        self.input_dim_capsule = input_shape[2]

        self.W = self.add_weight(shape=[self.num_capsules, self.input_num_capsules,
                                        self.dim_capsules, self.input_dim_capsule],
                                 initializer='glorot_uniform',
                                 name='W')

        self.built = True

    def call(self, inputs):
        inputs_expand = tf.expand_dims(tf.expand_dims(inputs, 1), -1)
        inputs_tiled = tf.tile(inputs_expand, [1, self.num_capsules, 1, 1, 1])
        inputs_hat = tf.map_fn(lambda x: tf.matmul(self.W, x), elems=inputs_tiled)

        b = tf.zeros(shape=[tf.shape(inputs)[0], self.num_capsules, self.input_num_capsules, 1, 1])

        assert self.routings > 0, 'The routings should be > 0.'
        for i in range(self.routings):
            c = tf.nn.softmax(b, axis=1)
            outputs = squash(tf.reduce_sum(c * inputs_hat, axis=2, keepdims=True))

            if i < self.routings - 1:
                b += tf.reduce_sum(inputs_hat * outputs, axis=-2, keepdims=True)

        return tf.squeeze(outputs, axis=[2, -1])

def squash(vectors, axis=-1):
    s_squared_norm = tf.reduce_sum(tf.square(vectors), axis=axis, keepdims=True)
    scale = s_squared_norm / (1 + s_squared_norm) / tf.sqrt(s_squared_norm + tf.keras.backend.epsilon())
    return scale * vectors


In [9]:
# Function to load data
def load_data(data_dir, img_size=(64, 64)):
    categories = ['deep', 'fast', 'normal']
    data = []
    labels = []
    for i, category in enumerate(categories):
        path = os.path.join(data_dir, category, f'RR_{category}')
        for img_name in os.listdir(path):
            if img_name.lower().endswith('.png'):
                img_path = os.path.join(path, img_name)
                img = load_img(img_path, target_size=img_size, color_mode='grayscale')
                img_array = img_to_array(img) / 255.0  # Normalize pixel values
                data.append(img_array)
                labels.append(i)
    return np.array(data), np.array(labels)

# Function to build the model
def build_model(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(256, 9, activation='relu', padding='valid')(inputs)
    x = layers.Conv2D(256, 9, activation='relu', padding='valid')(x)
    x = layers.Conv2D(128, 9, activation='relu', padding='valid')(x)
    x = layers.Conv2D(128, 9, activation='relu', padding='valid')(x)
    x = layers.Reshape((-1, 128))(x)
    digitcaps = CapsuleLayer(num_classes, 16)(x)
    output = layers.Lambda(lambda z: tf.sqrt(tf.reduce_sum(tf.square(z), axis=-1)))(digitcaps)

    model = models.Model(inputs=inputs, outputs=output)
    return model

In [5]:
!ls

drive  sample_data


In [ ]:
data_dir = "/content/drive/MyDrive/Data/images"

# Load the data
X, y = load_data(data_dir)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build and compile the model
model = build_model(input_shape=X.shape[1:], num_classes=3)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy}")

# Save the model
model.save('hr_prediction_model.h5')
print("Model saved as 'hr_prediction_model.h5'")

Epoch 1/50
 1/14 ━━━━━━━━━━━━━━━━━━━━ 14:42 68s/step - accuracy: 0.4375 - loss: 1.0899